In [1]:
import pandas as pd

In [44]:
import pandas as pd
import json

with open('/Users/jarana/workspace/projects/Entity-Attribute-Identification/data/queries.json') as f:
    jsonstr = json.load(f)

df = pd.DataFrame(jsonstr.items(), columns=['qid', 'text'])
df['type'] = ["query"] * len(df)


In [45]:
qrel = pd.read_csv('/Users/jarana/workspace/projects/Entity-Attribute-Identification/data/qrel.txt',names=["qid", "none", "property", "rele_label"], sep="\t")
qrel["property"] = [i.split(":")[1].replace(">","") for i in qrel.property.tolist()]

In [46]:
doc = pd.DataFrame(zip(["doc"] * len(qrel.property.unique()),[i for i in range(len(qrel.property.unique()))],qrel.property.unique()), columns=["type", "pid", "property"])

In [47]:
qrel = qrel.merge(doc[['pid', 'property']], how="left", on=["property"])

In [48]:
# Query Understanding via Entity Attribute Identification
# check Table2 for setting
sameType = ["INEX_XER", "TREC_Entity", "SemSearch_LS"]
df['qtype'] = [i.split("-")[0] if i.split("-")[0] not in sameType else "ListSerach" for i in df['qid']]
sameType = ["QALD2_te", "QALD2_tr"]
df['qtype'] = [i.split("-")[0] if i.split("-")[0] not in sameType else "QALD2" for i in df['qtype']]

In [50]:
df[['type', 'qid', 'text', 'qtype']].to_csv("/Users/jarana/workspace/WikiHow-Task-Based/data/cedr/eai-query.tsv", header=False, index=False, sep="\t")
qrel = qrel.merge(df[['qid','qtype']], on="qid", how="left")
qrel[['qid', 'none', 'pid', 'rele_label','qtype']].to_csv("/Users/jarana/workspace/WikiHow-Task-Based/data/cedr/eai-qrel.tsv", header=False, index=False, sep="\t")
doc.to_csv("/Users/jarana/workspace/WikiHow-Task-Based/data/cedr/eai-doc.tsv", header=False, index=False, sep="\t")

In [28]:
#5fold data generation
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import KFold
import random

foldNum = 5
trainId5fold = {i:[] for i in range(foldNum)}
validateId5fold = {i:[] for i in range(foldNum)}
testId5fold = {i:[] for i in range(foldNum)}

ids = qrel.qid.unique().tolist()
kf = KFold(n_splits=foldNum)
kf.get_n_splits(ids)
for idx, ele in enumerate(kf.split(ids)):
    train, test = [ids[i] for i in ele[0]], [ids[i] for i in ele[1]]
    train, val, _, _ = train_test_split(train, train, test_size=0.2)
    trainId5fold[idx].extend(train)
    validateId5fold[idx].extend(val)
    testId5fold[idx].extend(test)

def writeRunFile(filename, _df):
    for name, group in _df.groupby("qid"):
        for idx, row in enumerate(group.iterrows()):
            with open(filename, 'a') as fd:
                fd.write(str(row[1]['qid']) +"\tQ0\t" + str(row[1]['pid']) + "\t" + str(idx+1) + "\t" + str(row[1]['rele_label']) + "\trun\n")
                
cand_props = qrel.pid.unique()            
negNum = 3
for fold in range(foldNum):
    for idx, group in qrel[qrel.qid.isin(trainId5fold[fold])].groupby("qid"):
        pos = group['pid'].tolist()
        neg = list(set.difference(set(cand_props), set(pos)))
        print(len(neg), len(cand_props))
#         neg = random.sample(neg, len(pos) * negNum)
        
#         print(group.iloc[0].qid, df[df.qid == group.iloc[0].qid].text.values)
#         print(pos)
#         print(neg)
#         print()
        for idx, row in group.iterrows():
            with open("data/cedr/eai-train%d.tsv" % fold, 'a') as fd:
                fd.write(str(row["qid"]) + "\t"+str(row["pid"])+"\n")
        for n in neg:
            with open("data/cedr/eai-train%d.tsv" % fold, 'a') as fd:
                fd.write(str(row["qid"]) + "\t"+str(n)+"\n")
    
    for idx, group in qrel[qrel.qid.isin(validateId5fold[fold])].groupby("qid"):
        pos = group['pid'].tolist()
        neg = list(set.difference(set(cand_props), set(pos)))
#         neg = random.sample(neg, len(pos) * negNum)
        
        lines = {}
        for idx, row in group.iterrows():
            lines[str(row["qid"]) + "\tQ0\t"+str(row['pid'])+"\t0\t" +str(row['rele_label'])+"\trun\n"] = row['rele_label']

        for n in neg:
            lines[str(row["qid"]) + "\tQ0\t"+str(n)+"\t0\t0\trun\n"] = 0
        lines = [i[0] for i in sorted(lines.items(), key=lambda x: x[1], reverse=True)]
        for l in lines:
            with open("data/cedr/eai-valid%d.tsv" % fold, 'a') as fd:
                fd.write(l)
                
    for idx, group in qrel[qrel.qid.isin(testId5fold[fold])].groupby("qid"):
        pos = group['pid'].tolist()
        neg = list(set.difference(set(cand_props), set(pos)))
#         neg = random.sample(neg, len(pos) * negNum)
        lines = {}
        for idx, row in group.iterrows():
            lines[str(row["qid"]) + "\tQ0\t"+str(row['pid'])+"\t0\t" +str(row['rele_label'])+"\trun\n"] = row['rele_label']

        for n in neg:
            lines[str(row["qid"]) + "\tQ0\t"+str(n)+"\t0\t0\trun\n"] = 0
        lines = [i[0] for i in sorted(lines.items(), key=lambda x: x[1], reverse=True)]
        for l in lines:
            with open("data/cedr/eai-test%d.tsv" % fold, 'a') as fd:
                fd.write(l)
    

210 217
216 217
213 217
216 217
216 217
216 217
216 217
216 217
212 217
216 217
214 217
215 217
215 217
215 217
215 217
216 217
208 217
213 217
212 217
216 217
216 217
215 217
216 217
215 217
214 217
216 217
216 217
215 217
216 217
215 217
216 217
213 217
215 217
215 217
216 217
215 217
214 217
216 217
214 217
216 217
212 217
216 217
216 217
215 217
215 217
216 217
214 217
213 217
216 217
215 217
216 217
216 217
213 217
216 217
216 217
216 217
215 217
215 217
206 217
213 217
216 217
216 217
216 217
215 217
215 217
213 217
215 217
215 217
214 217
215 217
212 217
215 217
215 217
214 217
216 217
216 217
216 217
215 217
215 217
216 217
214 217
216 217
212 217
216 217
216 217
215 217
215 217
215 217
216 217
215 217
215 217
207 217
216 217
216 217
216 217
213 217
215 217
211 217
216 217
214 217
215 217
214 217
211 217
214 217
215 217
214 217
215 217
216 217
211 217
215 217
216 217
216 217
214 217
214 217
213 217
215 217
215 217
210 217
215 217
216 217
209 217
214 217
215 217
216 217
216 217


In [6]:
import wikipedia
from nltk.corpus import stopwords
import re, nltk, random, os, json

stop_words = set(stopwords.words('english'))

regex = re.compile('[^a-zA-Z0-9.,]')
#First parameter is the replacement, second parameter is your input string
def preprocessingText(doc,enableStopword=False):
    doc = regex.sub(' ', doc)
    if enableStopword:
        doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc.lower()

queries = df.text.tolist()
qids = df.qid.tolist()
for qid, text in zip(qids, queries):
    try:
        with open("data/cedr/eai-wikipedia.tsv", 'a') as fd:
            fd.write("query\t"+str(qid)+"\t"+preprocessingText(wikipedia.summary(wikipedia.search(text)[0]))+"\n")
    except:
        with open("data/cedr/eai-wikipedia.tsv", 'a') as fd:
            fd.write("query\t"+str(qid)+"\t"+str(text)+"\n")
#     print(wikipedia.summary())
#     print()

/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [4]:
from run_scraper import MultiThreadScraper
# wikihows = pd.read_csv("data/cedr/wikihow.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()
# title_to_crawl = [i.split(".")[0] for i in wikihows]
s = MultiThreadScraper(df.text.tolist(), "data/eai-questions/", "fail")
s.run_scraper()


100%|██████████| 167/167 [00:00<00:00, 6476.46it/s]


Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)

Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)

Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)

Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)

Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)

Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)

Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)

Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)

Message: element not interactable
  (Session info: headless chrome=80.0.3987.100)



In [9]:
with open("data/cedr/eai-question-qq.tsv", 'a') as fd:

    for qid, text in zip(qids, queries):

        try:
            lines = open("data/eai-questions/%s.csv" % text.replace(" ", "_")).read().splitlines()[1:]
        except:
            fd.write("question\t"+str(qid)+"\t"+text+"\n")
            continue
        questions = []
        for i in lines[1:]:
            questions.extend(i.split(";"))
        questions = " ".join(list(set(questions)))
        if questions == "":
            fd.write("question\t"+str(qid)+"\t"+text+"\n")
        else:
            fd.write("question\t"+str(qid)+"\t"+ questions +"\n")


In [138]:
import collections
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG, RR, AP
qrelDict = collections.defaultdict(dict)
for qid, prop, label in qrel[['qid', 'pid', 'rele_label']].values:
    qrelDict[str(qid)][str(prop)] = int(label)
    
def evaluate(qrels, ranked_list):
    res = []
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)
    result = {}
    
    
    
    for i in [5, 10 ,15, 20]:
        metric = MSnDCG(xrelnum, grades, cutoff=i)
        result["ndcg@%d" % i] = metric.compute(labeled_ranked_list)
        
        nerr = nERR(xrelnum, grades, cutoff=i)
        result["nerr@%d" % i] = nerr.compute(labeled_ranked_list)
        
        
        _ranked_list = ranked_list[:i]
        result["p@%d" % i] = len(set.intersection(set(qrels.keys()), set(_ranked_list))) / len(_ranked_list)
        result["r@%d" % i] = len(set.intersection(set(qrels.keys()), set(_ranked_list))) / len(qrels)
    result["rp"] = len(set.intersection(set(qrels.keys()), set(ranked_list[:len(qrels)]))) / len(qrels)
    map = AP(xrelnum, grades)
    result["map"] = map.compute(labeled_ranked_list)
    mrr = RR()
    result["mrr"] = mrr.compute(labeled_ranked_list)
    return result

In [169]:
df_run = pd.read_csv("/Users/jarana/workspace/projects/Entity-Attribute-Identification/data/runs/MRF/MRF_output_lm1_0_6_lm2_0_2.run", names=["qid", "none", "pid", "rank", "score", "none2"], sep="\t")
df_run["pid"] = [i.split(":")[1].replace(">","") for i in df_run.pid.tolist()]

res = []
for fold in range(5):
    _df = pd.read_csv("data/cedr/eai-test%d.tsv" %fold, sep="\t", usecols=[0], names=["qid"])
    testIds = _df.qid.unique()

    for tid in testIds:
        ranked_list = df_run[df_run.qid == tid].pid.tolist()
        if len(ranked_list) == 0:
            continue
        randomList = qrel.pid.unique().tolist()
        random.shuffle(randomList)
        base = evaluate(qrelDict[tid], randomList)
        res.append(base)

keys = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg", "nerr"] for j in [5, 10 ,15, 20]}
keys["rp"] = []
keys["mrr"] = []
keys["map"] = []

for r in [res]:
    for k in keys:
        print(np.mean([i[k] for i in r]), end="\t")
    print()

0.014634146341463417	0.010365853658536586	0.010975609756097562	0.01128048780487805	0.027383592017738354	0.037749445676274945	0.06000818287403653	0.08882641748495407	0.018687421796560442	0.022517671367779112	0.029826336617913993	0.03718407382298806	0.016828691129437096	0.019100880552141073	0.021939927538525732	0.02391896713551566	0.013710273466371025	0.04948062868814621	0.03417804520926004	


In [84]:
df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['headline'] = df_wiki['headline'].str.replace("\n", "")
df_wiki['title'] = df_wiki['title'].str.replace("How to ", "")
for col in ['overview', 'headline', 'text', 'sectionLabel', 'title']:
    df_wiki[col] = [preprocessingText(str(i), False) for i in df_wiki[col]]
    
df_wiki['title'] = [i if not i[-1].isdigit() else i[:-1] for i in df_wiki['title']]

In [192]:
qrel['cat'] = [i.split("-")[0] for i in qrel['qid'].tolist()]

In [203]:
df.groupby("cat2").size()

cat2
INEX_LD         31
INEX_XER        13
QALD2_te        28
QALD2_tr        34
SemSearch_ES    40
SemSearch_LS    14
TREC_Entity      7
dtype: int64

In [209]:
qrel

,qid,none,pid,rele_label,cat,text,type
0,INEX_LD-2009039,Q0,capital,1,INEX_LD,roman architecture,query
1,INEX_LD-2009039,Q0,governmentType,1,INEX_LD,roman architecture,query
2,INEX_LD-2009061,Q0,place,1,INEX_LD,france second world war normandy,query
3,INEX_LD-2009063,Q0,commander,1,INEX_LD,D Day normandy invasion,query
4,INEX_LD-2009063,Q0,isPartOfMilitaryConflict,2,INEX_LD,D Day normandy invasion,query
5,INEX_LD-2009063,Q0,place,1,INEX_LD,D Day normandy invasion,query
6,INEX_LD-2009063,Q0,territory,1,INEX_LD,D Day normandy invasion,query
7,INEX_LD-2009063,Q0,combatant,1,INEX_LD,D Day normandy invasion,query
8,INEX_LD-2009063,Q0,partof,2,INEX_LD,D Day normandy invasion,query
9,INEX_LD-2009096,Q0,mainContractor,2,INEX_LD,Eiffel,query


In [201]:
df['cat2'] = [i.split("-")[0] for i in df['qid'].tolist()]

In [197]:
qrel = qrel.merge(df, on="qid", how="left")

In [208]:
qrel[qrel.cat == "INEX_LD"]

,qid,none,pid,rele_label,cat,text,type
0,INEX_LD-2009039,Q0,capital,1,INEX_LD,roman architecture,query
1,INEX_LD-2009039,Q0,governmentType,1,INEX_LD,roman architecture,query
2,INEX_LD-2009061,Q0,place,1,INEX_LD,france second world war normandy,query
3,INEX_LD-2009063,Q0,commander,1,INEX_LD,D Day normandy invasion,query
4,INEX_LD-2009063,Q0,isPartOfMilitaryConflict,2,INEX_LD,D Day normandy invasion,query
5,INEX_LD-2009063,Q0,place,1,INEX_LD,D Day normandy invasion,query
6,INEX_LD-2009063,Q0,territory,1,INEX_LD,D Day normandy invasion,query
7,INEX_LD-2009063,Q0,combatant,1,INEX_LD,D Day normandy invasion,query
8,INEX_LD-2009063,Q0,partof,2,INEX_LD,D Day normandy invasion,query
9,INEX_LD-2009096,Q0,mainContractor,2,INEX_LD,Eiffel,query


In [189]:
len(qrel) / 16

25.9375

In [86]:
from rank_bm25 import BM25Okapi
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

corpus = []
doc2title = {}
for name, group in df_wiki.groupby("title"):
    doc = []
    for col in ["title", "overview", "sectionLabel", "headline", "text"]:
        doc.append(" ".join(group[col].unique().tolist()))
        
    doc = "\t".join(doc)
    corpus.append(doc)
    doc2title[doc] = name

In [88]:
bm25 = BM25Okapi([tokenizer.tokenize(doc) for doc in corpus])

In [126]:
with open("data/cedr/eai-wikihow.tsv", 'a') as fd:
    for idx, row in df.iterrows():
        doc = bm25.get_top_n(tokenizer.tokenize(row['text']), corpus, n=1)[0]
        doc = doc.replace("\t", " ")
        fd.write("query\t"+str(row['qid'])+"\t"+doc+"\n")
#         break

In [109]:
import pickle

In [129]:
import urllib, time
import http.client, urllib.request, urllib.parse, urllib.error, base64
import time, os, pickle

def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '924c1505854b4da4a6144a1cce92937f',
    }
    queries = [str(i).replace("\'", "") for i in queries]
    params = urllib.parse.urlencode({})
    
    while True:
        try:
            conn = http.client.HTTPSConnection('api.msturing.org')
            conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
            response = conn.getresponse()
            data = response.read()
            data = json.loads(data)
            conn.close()
            break
        except Exception as e:
            print(e)
            time.sleep(5)
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

def getTermMSvec(all_properties):
    tmp = {}
    for i in range(0, len(all_properties), 20):
        data = getVectors(all_properties[i:i+20])
        for i in data:
            tmp[i] = data[i]
    return tmp
def filterText(docs, maxlen):
    return [" ".join(i.split(" ")[:maxlen]) for i in docs]

def saveDict(d, name):
    f = open(name,"wb")
    pickle.dump(d,f)
    f.close()

for maxlen in [32, 64, 128]:
    queries = [i if i[-1] != " " else i[:-1] for i in pd.read_csv("data/cedr/eai-query.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()]
    properties = pd.read_csv("data/cedr/eai-doc.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()
    wikipedias = pd.read_csv("data/cedr/eai-wikipedia.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()
    questions = pd.read_csv("data/cedr/eai-question-qq.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()
    wikihows = pd.read_csv("data/cedr/eai-wikihow.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()
    
    wikipedias = filterText(wikipedias, maxlen)
    questions = filterText(questions, maxlen)
    wikihows = filterText(wikihows, maxlen)
    
    texts = queries + properties + wikipedias + questions + wikihows
    texts = [i.replace("\'", "") for i in texts]
    data = getTermMSvec(texts)
    saveDict(data, "data/cedr/eai-ms%d" % maxlen)
    

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


In [162]:
max([len(tokenizer.tokenize(i)) for i in wikihows])

173